# Predictions of Druggable peptides using the best model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# remove warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix,accuracy_score, roc_auc_score,f1_score, recall_score, precision_score
from sklearn.utils import class_weight

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import LinearSVC

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_selection import RFECV, VarianceThreshold, SelectKBest, chi2
from sklearn.feature_selection import SelectFromModel, SelectPercentile, f_classif

import seaborn as sns; sns.set() # data visualization library 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from imblearn.over_sampling import SMOTE

from sklearn.datasets import load_iris
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, StratifiedKFold
import numpy as np

import pandas as pd
from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.externals import joblib

print(__doc__)

Automatically created module for IPython interactive environment


Using TensorFlow backend.
C:\Users\Eric\Miniconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
def DataCheckings(df):
    # CHECKINGS ***************************
    # Check the number of data points in the data set
    print("\nData points =", len(df))
    
    # Check the number of columns in the data set
    print("\nColumns (output + features)=",len(df.columns))
    
    # Check the data types
    print("\nData types =", df.dtypes.unique())
    
    # Dataset statistics
    print('\n')
    df.describe()
    
    # print names of columns
    print('Column Names:\n', df.columns)
    
    # see if there are categorical data
    print("\nCategorical features:", df.select_dtypes(include=['O']).columns.tolist())
    
    # Check NA values
    # Check any number of columns with NaN
    print("\nColumns with NaN: ", df.isnull().any().sum(), ' / ', len(df.columns))

    # Check any number of data points with NaN
    print("\nNo of data points with NaN:", df.isnull().any(axis=1).sum(), ' / ', len(df))

In [3]:
def getDataFromDataset(sFile, OutVar):
    # read details file
    print('\n-> Read dataset', sFile)
    df = pd.read_csv(sFile)
    #df = feather.read_dataframe(sFile)
    
    DataCheckings(df)
    
    # remove duplicates!
    df.drop_duplicates(keep=False, inplace=True)
    
    print('Shape', df.shape)
    # print(list(df.columns))

    # select X and Y
    ds_y = df[OutVar]
    ds_X = df.drop(OutVar,axis = 1)
    Xdata = ds_X.values # get values of features
    Ydata = ds_y.values # get output values

    print('Shape X data:', Xdata.shape)
    print('Shape Y data:',Ydata.shape)
    
    # return data for X and Y, feature names as list
    return (Xdata, Ydata, list(ds_X.columns))

In [4]:
def  set_weights(y_data, option='balanced'):
    """Estimate class weights for umbalanced dataset
       If ‘balanced’, class weights will be given by n_samples / (n_classes * np.bincount(y)). 
       If a dictionary is given, keys are classes and values are corresponding class weights. 
       If None is given, the class weights will be uniform """
    cw = class_weight.compute_class_weight(option, np.unique(y_data), y_data)
    w = {i:j for i,j in zip(np.unique(y_data), cw)}
    return w 

In [5]:
# define output variables
outVar = 'Class'

# define list of folds
foldType = 3

# define a label for output files
targetName = 'GS_Outer'

seed = 28

## Reproduce the pipeline model

In [6]:
sFile = r'C:\Users\Eric\Documents\GitHub\machine-learning-for-druggable-proteins\datasets\ds.Class_TC_ballanced.csv'
# get data from file
Xdata, Ydata, Features = getDataFromDataset(sFile,outVar) # n_sample=100


-> Read dataset C:\Users\Eric\Documents\GitHub\machine-learning-for-druggable-proteins\datasets\ds.Class_TC_ballanced.csv

Data points = 1332

Columns (output + features)= 8001

Data types = [dtype('float64') dtype('int64')]


Column Names:
 Index(['AAA', 'RAA', 'NAA', 'DAA', 'CAA', 'EAA', 'QAA', 'GAA', 'HAA', 'IAA',
       ...
       'KVV', 'MVV', 'FVV', 'PVV', 'SVV', 'TVV', 'WVV', 'YVV', 'VVV', 'Class'],
      dtype='object', length=8001)

Categorical features: []

Columns with NaN:  0  /  8001

No of data points with NaN: 0  /  1332
Shape (1332, 8001)
Shape X data: (1332, 8000)
Shape Y data: (1332,)


In [7]:
# Calculate class weights
class_weights = set_weights(Ydata)
print("Class weights = ", class_weights)

Class weights =  {0: 1.0, 1: 1.0}


In [8]:
outer_cv = StratifiedKFold(n_splits=3,shuffle=True,random_state=seed)

In [9]:
ifold = 0
ACCs  =[]
AUROCs=[]
models =[]
SelectedFeatures =[]

for train_index, test_index in outer_cv.split(Xdata, Ydata):
    ifold +=1
    
    print("Fold =",ifold)
    start = time.time()
    
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = Xdata[train_index], Xdata[test_index]
    y_train, y_test = Ydata[train_index], Ydata[test_index]
    
    # Standardize dataset
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test  = scaler.transform(X_test)
    
    # Feature selection # FS = SelectFromModel(LinearSVC(), max_features = 400,threshold=-np.inf)
    lsvc = LinearSVC(max_iter=50000).fit(X_train, y_train)
    model = SelectFromModel(lsvc, prefit=True,max_features = 200,threshold=-np.inf)
    X_train = model.transform(X_train)
    X_test  = model.transform(X_test)
    #print("Selected X:", X_train.shape)

    # Selected features
    SelFeatures = []
    for i in model.get_support(indices=True):
        SelFeatures.append(Features[i])
    SelectedFeatures.append(SelFeatures)

    #scaler.transform(X_test)
    clf = SVC(kernel = 'rbf', random_state=seed,gamma='scale',
              class_weight=class_weights,probability=True)
    clf.fit(X_train, y_train)
    
    joblib.dump(clf, 'SVM_model'+str(ifold)+'.pkl', compress = 1)
    models.append(clf)
    
    y_pred = clf.predict_proba(X_test)
    AUROC = roc_auc_score(y_test, y_pred[:, 1])
    AUROCs.append(AUROC)
    
    ACC = clf.score(X_test,y_test)
    ACCs.append(ACC)
   
    print("AUROC=",AUROC,"ACC=",ACC, (time.time() - start)/60,"mins")
    

Fold = 1
AUROC= 0.9620972323675026 ACC= 0.9234234234234234 1.2529455900192261 mins
Fold = 2
AUROC= 0.971370018667316 ACC= 0.9324324324324325 1.343106253941854 mins
Fold = 3
AUROC= 0.964572680788897 ACC= 0.9279279279279279 1.721213686466217 mins


Let's see the mean AUROC values for best model and the standard deviations:

In [10]:
print(np.mean(AUROCs),np.std(AUROCs))

0.9660133106079053 0.003920263779140678


In [11]:
print(np.mean(ACCs),np.std(ACCs))

0.9279279279279279 0.0036779125267014765


In [12]:
# all the selected features for the 3 folds
print(SelectedFeatures)

[['NRA', 'INA', 'KCA', 'YEA', 'LGA', 'THA', 'IMA', 'YPA', 'CSA', 'ARR', 'KNR', 'WDR', 'TER', 'AQR', 'PQR', 'PPR', 'FWR', 'NYR', 'ERN', 'YRN', 'SDN', 'RHN', 'YIN', 'FWN', 'TWN', 'CDD', 'MED', 'LGD', 'SHD', 'PKD', 'WPD', 'SSD', 'NTD', 'HTD', 'DWD', 'EYD', 'VYD', 'QRC', 'YGC', 'NHC', 'IHC', 'VHC', 'EMC', 'GMC', 'GFC', 'QSC', 'GYC', 'EVC', 'DNE', 'GDE', 'KDE', 'NHE', 'ALE', 'RLE', 'HME', 'TSE', 'AWE', 'KWE', 'EYE', 'QYE', 'GVE', 'FVE', 'SAQ', 'TRQ', 'INQ', 'FNQ', 'PCQ', 'WEQ', 'RQQ', 'HLQ', 'RMQ', 'DFQ', 'HFQ', 'DSQ', 'YSQ', 'RVQ', 'NAG', 'HGG', 'QHG', 'NKG', 'RVG', 'TNH', 'FDH', 'PDH', 'TQH', 'DGH', 'CGH', 'KHH', 'KKH', 'WSH', 'FTH', 'FWH', 'CYH', 'TYH', 'FEI', 'WEI', 'WQI', 'QGI', 'DRL', 'IRL', 'PDL', 'NGL', 'IKL', 'DPL', 'GPL', 'ESL', 'VWL', 'DVL', 'MVL', 'QRK', 'GRK', 'HNK', 'YNK', 'MDK', 'HCK', 'DHK', 'QLK', 'MMK', 'SMK', 'FFK', 'EWK', 'TYK', 'WRM', 'WCM', 'REM', 'WQM', 'AGM', 'SHM', 'LLM', 'YLM', 'NFM', 'TPM', 'TSM', 'VTM', 'RWM', 'KYM', 'IVM', 'YVM', 'LDF', 'YQF', 'YKF', 'CFF', 'GFF

In [13]:
# differences of selected descriptors
print(list(set(SelectedFeatures[1])-set(SelectedFeatures[0])))

['ETP', 'NDQ', 'HSV', 'WNM', 'KQQ', 'SCD', 'QNI', 'LVH', 'EKL', 'YVL', 'VGF', 'YWD', 'ACD', 'VGW', 'SDE', 'VSR', 'CDM', 'NDC', 'AWG', 'QTI', 'IMV', 'FAP', 'DIW', 'TGY', 'SFG', 'SPI', 'TRY', 'GPQ', 'QFH', 'WHV', 'VFD', 'QWD', 'MDN', 'LHN', 'WAA', 'FWC', 'HDD', 'AFY', 'WLD', 'WHC', 'INT', 'VYA', 'DQN', 'HTV', 'QWT', 'ESI', 'PMI', 'ENV', 'GYM', 'RGD', 'RNT', 'HKP', 'IVG', 'CDH', 'VHP', 'RGW', 'YIA', 'CRW', 'EEA', 'NII', 'HVM', 'KPH', 'RTG', 'WGS', 'GKW', 'VVL', 'FYT', 'TYF', 'EHR', 'PIP', 'GEW', 'FEM', 'MMA', 'KCW', 'AQA', 'GTI', 'CHQ', 'WLT', 'LLV', 'YRY', 'LYP', 'PSS', 'KGG', 'AEK', 'AYI', 'GIG', 'CDS', 'ENQ', 'RSN', 'QRG', 'QYQ', 'DLT', 'AIF', 'PTG', 'PAI', 'SMM', 'QTA', 'GGG', 'HDS', 'YDH', 'VLD', 'SSG', 'WRC', 'DSA', 'NGQ', 'QAT', 'LTD', 'WRI', 'DHG', 'MMF', 'AFF', 'HLA']


In [14]:
# differences of selected descriptors
print(list(set(SelectedFeatures[1])-set(SelectedFeatures[2])))

['ETP', 'FNP', 'NDQ', 'HSV', 'CFF', 'KQQ', 'SCD', 'QNI', 'YVL', 'VGF', 'YWD', 'LLM', 'ACD', 'VGW', 'SDE', 'VSR', 'KKT', 'CDM', 'QTI', 'AWG', 'IMV', 'FAP', 'DIW', 'YSQ', 'PDL', 'SFG', 'SPI', 'TRY', 'NLT', 'GPQ', 'QFH', 'WHV', 'VFD', 'QWD', 'LHN', 'HNK', 'WAA', 'FWC', 'HDD', 'AFY', 'YAV', 'WLD', 'WHC', 'PET', 'DQN', 'HTV', 'FFK', 'FWF', 'ESI', 'ENV', 'TYK', 'GYM', 'RGD', 'RNT', 'HKP', 'RVQ', 'WQI', 'CDH', 'VHP', 'LMY', 'YIA', 'INQ', 'CRW', 'EEA', 'NII', 'HVM', 'YKF', 'KPH', 'RWM', 'WGS', 'DWD', 'TYF', 'FYT', 'TGV', 'PIP', 'INA', 'GEW', 'KCA', 'TSE', 'MMA', 'KCW', 'AQA', 'GTI', 'CHQ', 'WLT', 'EMC', 'YRY', 'LYP', 'PSS', 'KGG', 'AEK', 'GVE', 'AYI', 'GIG', 'RSP', 'CDS', 'ENQ', 'PLP', 'RSN', 'YQF', 'QYQ', 'DLT', 'ESL', 'AIF', 'PTG', 'PAI', 'SMM', 'QTA', 'GGG', 'HDS', 'DPL', 'IMA', 'VLD', 'SSG', 'RCW', 'WRC', 'DSA', 'FWH', 'QAT', 'FWN', 'LTD', 'WRI', 'DHG', 'MMF', 'IKL', 'HLA']


## Predictions with the best model

We choose model 2 as the best due to the maximum ACC value (AUROC= 0.9752, ACC= 0.937).

Load the prediction datasets (the same format as the dataset: 8000 TC features + Class=-1):

In [40]:
cdca3 = """MGSAKSVPVTPARPPPHNKHLARVADPRSPSAGILRTPIQVESSPQPGLPAGEQLEGLKH
AQDSDPRSPTLGIARTPMKTSSGDPPSPLVKQLSEVFETEDSKSNLPPEPVLPPEAPLSS
ELDLPLGTQLSVEEQMPPWNQTEFPSKQVFSKEEARQPTETPVASQSSDKPSRDPETPRS
SGSMRNRWKPNSSKVLGRSPLTILQDDNSPGTLTLRQGKRPSPLSENVSELKEGAILGTG
RLLKTGGRAWEQGQDHDKENQHFPLVES"""

header = open("example_header.csv", "r").readlines()[0].split(",")


counts = []


for x in header:
    count = cdca3.count(x)
    counts.append(count)

fractions = [c/len(counts[:-1]) for c in counts]

fractions[-1] = -1
    
import pandas as pd
import numpy as np


df = pd.DataFrame(np.array([fractions]), columns=header)

df.to_csv("cdca3.csv", index=False)



# get data from files and check the files
sFile1 = r'C:\Users\Eric\Documents\GitHub\machine-learning-for-druggable-proteins\cdca3.csv'
Xdata1, Ydata1, Features1 = getDataFromDataset(sFile1,outVar) 

print(Xdata1.shape)


-> Read dataset C:\Users\Eric\Documents\GitHub\machine-learning-for-druggable-proteins\cdca3.csv

Data points = 1

Columns (output + features)= 8001

Data types = [dtype('float64')]


Column Names:
 Index(['AAA', 'RAA', 'NAA', 'DAA', 'CAA', 'EAA', 'QAA', 'GAA', 'HAA', 'IAA',
       ...
       'KVV', 'MVV', 'FVV', 'PVV', 'SVV', 'TVV', 'WVV', 'YVV', 'VVV', 'Class'],
      dtype='object', length=8001)

Categorical features: []

Columns with NaN:  0  /  8001

No of data points with NaN: 0  /  1
Shape (1, 8001)
Shape X data: (1, 8000)
Shape Y data: (1,)
(1, 8000)


Use only the second split / model - scale the prediction datasets, select only the features of model 2, predict the class and predict the probability of that class:

In [41]:
ifold = 0

for train_index, test_index in outer_cv.split(Xdata, Ydata):
    ifold +=1
    
    if ifold ==2: # only model 2
        print("Fold =",ifold)
        start = time.time()

        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = Xdata[train_index], Xdata[test_index]
        y_train, y_test = Ydata[train_index], Ydata[test_index]

        # Standardize dataset
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test  = scaler.transform(X_test)
        # scale prediction set
        Xdata1  = scaler.transform(Xdata1)

        
        # Feature selection # FS = SelectFromModel(LinearSVC(), max_features = 400,threshold=-np.inf)
        lsvc = LinearSVC(max_iter=50000).fit(X_train, y_train)
        model = SelectFromModel(lsvc, prefit=True,max_features = 200,threshold=-np.inf)
        X_train = model.transform(X_train)
        X_test  = model.transform(X_test)
    
        # Selected features
        SelFeatures = []
        for i in model.get_support(indices=True):
            SelFeatures.append(Features[i])
        
        # apply selected features to prediction set
        Xdata1 = Xdata1[:, model.get_support()]
        print("Xdata1 sel=", Xdata1.shape)


        # we dont need to calculate again, but load the model from the disk!
        #clf = SVC(kernel = 'rbf', random_state=seed,gamma='scale',
        #          class_weight=class_weights,probability=True)
        #clf.fit(X_train, y_train)

        # load the saved model from disk
        clf = joblib.load('SVM_model'+str(ifold)+'.pkl')
        #joblib.dump(clf, 'SVM_model'+str(ifold)+'.pkl', compress = 1)
        
        # predictions with the model
        Ydata1 = clf.predict(Xdata1)

        # add probabilities (n_samples X n_classes; class 0, class 1)
        Ydata1prob = clf.predict_proba(Xdata1)

        # save predictions for list 1
        dff1 = pd.DataFrame(Xdata1,columns=SelFeatures)
        dff1['Class'] = Ydata1
        dff1['Prob0'] = Ydata1prob[:,0]
        dff1['Prob1'] = Ydata1prob[:,1]
        
        # merge with protein information from other file
        #result = pd.concat([dff1, pd.read_csv('./PREDICTIONS/TC_seqs.Screening_1_Cancer_Driver_Genes.csv')], axis=1)
        # creat new order of columns in final results
       # newHeader=['Class','Prob1','Prob0','V1','V2']
        #result = result[newHeader]
       # result = result.sort_values(by=['Prob1'], ascending=False)
        dff1.to_csv(sFile1[:-4]+'_predictions.csv', index=True)


        print("Time",(time.time() - start)/60,"mins")

Fold = 2
Xdata1 sel= (1, 200)
Time 1.0893101930618285 mins


In [18]:
print("==> Chekc the results:")
print(sFile1[:-4]+'_predictions.csv')


==> Chekc the results:
./datasets/ds.Screening_1_TC_predictions.csv


NameError: name 'sFile2' is not defined

Hf with ML!@muntisa